## 7 tensorflow的高级框架

## 7.1 TFLearn

### 7.1.1 加载数据

In [4]:
import tflearn

In [5]:
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression
import tflearn.datasets.oxflower17 as oxflower17
X, Y = oxflower17.load_data(one_hot=True, resize_pics=(227, 227))

### 7.1.2 构建网络

In [3]:
# 构建AlexNet网络
network = input_data(shape=[None, 227, 227, 3])
network = conv_2d(network, 96, 11, strides=4, activation='relu')
network = max_pool_2d(network, 3, strides=2)
network = local_response_normalization(network)
network = conv_2d(network, 256, 5, activation='relu')
network = max_pool_2d(network, 3, strides=2)
network = local_response_normalization(network)
network = conv_2d(network, 384, 3, activation='relu')
network = conv_2d(network, 384, 3, activation='relu')
network = conv_2d(network, 256, 3, activation='relu')
network = max_pool_2d(network, 3, strides=2)
network = local_response_normalization(network)
network = fully_connected(network, 4096, activation='tanh')
network = dropout(network, 0.5)
network = fully_connected(network, 4096, activation='tanh')
network = dropout(network, 0.5)
network = fully_connected(network, 17, activation='softmax')
network = regression(network, optimizer='momentum', loss='categorical_crossentropy', learning_rate=0.001)

### 7.1.3 训练模型

In [ ]:
model = tflearn.DNN(network, checkpoint_path='model_alexnet', max_checkpoints=1, tensorboard_verbose=2)
model.fit(X, Y, n_epoch=1000, validation_set=0.1, shuffle=True,
          show_metric=True, batch_size=64, snapshot_step=200,
          snapshot_epoch=False, run_id='alexnet_oxflowers17')

Training Step: 21  | total loss: 2.82188 | time: 3.841s
| Momentum | epoch: 002 | loss: 2.82188 - acc: 0.1085 -- iter: 0064/1224


## 7.2 Keras

### 7.2.1 Keras 的模型

In [8]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation
model = Sequential()
model.add(Dense(output_dim=64, input_dim=100))
model.add(Activation("relu"))
model.add(Dense(output_dim=10))
model.add(Activation("softmax"))

/Users/fatu/anaconda/envs/tensorflowpy2/lib/python2.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=64, input_dim=100)`
  """
/Users/fatu/anaconda/envs/tensorflowpy2/lib/python2.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=10)`
  import sys


### Keras 的使用

1. 实现卷积神经网络

In [12]:
from __future__ import print_function
import keras
from keras import utils as np_utils
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [26]:
batch_size = 128
num_classes = 10
epochs = 12

In [27]:
# 输入图片维度
img_rows, img_cols = 28, 28
# 卷积滤镜的个数
nb_filters = 32
# 最大池化，池化核大小
pool_size = (2, 2)
# 卷积核大小
kernel_size = (3, 3)

In [28]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [29]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# 将类向量转换为二进制类矩阵
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [32]:
# 用2个卷积层、1个池化层和2个全连接层来构建
model = Sequential()
model.add(Conv2D(nb_filters, kernel_size=kernel_size,
                       activation='relu',
                       input_shape=input_shape))
model.add(Conv2D(nb_filters, kernel_size,activation='relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [34]:
# 用model.compile()函数编译模型，采用多分类的损失函数，用Adadelta算法做优化方法
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [35]:
# 然后，开始用model.fit()函数训练模型，输入训练集和测试数据，以及batch_size和epoch参数
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))


Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 3s - loss: 0.3895 - acc: 0.8812 - val_loss: 0.0995 - val_acc: 0.9694
Epoch 2/12
60000/60000 [==============================] - 3s - loss: 0.1465 - acc: 0.9568 - val_loss: 0.0673 - val_acc: 0.9788
Epoch 3/12
60000/60000 [==============================] - 3s - loss: 0.1111 - acc: 0.9674 - val_loss: 0.0541 - val_acc: 0.9819
Epoch 4/12
60000/60000 [==============================] - 3s - loss: 0.0929 - acc: 0.9731 - val_loss: 0.0459 - val_acc: 0.9851
Epoch 5/12
60000/60000 [==============================] - 3s - loss: 0.0782 - acc: 0.9766 - val_loss: 0.0432 - val_acc: 0.9847
Epoch 6/12
60000/60000 [==============================] - 3s - loss: 0.0717 - acc: 0.9788 - val_loss: 0.0393 - val_acc: 0.9865
Epoch 7/12
60000/60000 [==============================] - 3s - loss: 0.0650 - acc: 0.9815 - val_loss: 0.0366 - val_acc: 0.9882
Epoch 8/12
60000/60000 [==============================] - 3s 

In [37]:
# 最后用model.evaluat()函数来评估模型
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.0299538767546
Test accuracy: 0.9898


2. 模型的加载及保存

In [38]:
from keras.models import save_model, load_model

In [39]:
def test_sequential_model_saving():
    model = Sequential()
    model.add(Dense(2, input_shape=(3,)))
    model.add(RepeatVector(3))
    model.add(TimeDistributed(Dense(3)))
    model.compile(loss=losses.MSE,
                  optimizer=optimizers.RMSprop(lr=0.0001),
                  metrics=[metrics.categorical_accuracy],
                  sample_weight_mode='temporal')
    x = np.random.random((1, 3))
    y = np.random.random((1, 3, 3))
    model.train_on_batch(x, y)

    out = model.predict(x)
    _, fname = tempfile.mkstemp('.h5')
    save_model(model, fname)

    new_model = load_model(fname)
    os.remove(fname)

    out2 = new_model.predict(x)
    assert_allclose(out, out2, atol=1e-05)

    # test that new updates are the same with both models
    x = np.random.random((1, 3))
    y = np.random.random((1, 3, 3))
    model.train_on_batch(x, y)
    new_model.train_on_batch(x, y)
    out = model.predict(x)
    out2 = new_model.predict(x)
    assert_allclose(out, out2, atol=1e-05)

In [40]:
json_string = model.to_json()
json_string = model.to_yaml()

保存完成后，还可以手动编辑，并且使用如下语句进行加载：

如果仅需要保存模型权重，不包含模型结构，可以使用save_weights和load_weights语句来保存和加载:

In [42]:
model.save_weights('my_model_weights.h5')
model.load_weights('my_model_weights.h5')